# Coadd script

In [39]:
# imports
import glob
import pdb

from astropy.table import Table, vstack

from xastropy.xutils import xdebug as xdb

## Testing

In [4]:
x1d_path = os.getenv('DROPBOX_DIR')+'/COS-LRG/tmp/'

In [6]:
x1d_files = glob.glob(x1d_path+'*x1d.fits')

### Check an x1d file

In [9]:
tbl = Table.read(x1d_files[0])

In [15]:
tbl[0:1]

SEGMENT,EXPTIME,NELEM,WAVELENGTH [16384],FLUX [16384],ERROR [16384],GROSS [16384],GCOUNTS [16384],NET [16384],BACKGROUND [16384],DQ [16384],DQ_WGT [16384]
,s,,Angstrom,erg / (Angstrom cm2 s),erg / (Angstrom cm2 s),ct / s,ct,ct / s,ct / s,,
str4,float64,int32,float64,float32,float32,float32,float32,float32,float32,int16,float32
FUVA,1128.192,16384,1185.08305117 .. 2496.14189625,0.0 .. 0.0,2.04881e-16 .. 6.06116e-17,0.0 .. 0.0,0.0 .. 0.0,0.0 .. 0.0,0.0 .. 0.0,128 .. 128,0.0 .. 0.0


### Segment A

In [20]:
# Load
sega_tbls = []
for x1d_file in x1d_files:
    tbl = Table.read(x1d_file)
    sega_tbls.append(tbl[0:1])

In [31]:
# Grab one wavelength array
wave = sega_tbls[0]['WAVELENGTH'][0,:].data
wave[0:5]

array([ 1185.08305117,  1185.16248733,  1185.24192356,  1185.32135985,
        1185.40079622])

In [37]:
# Sum exposure time
total_time = np.zeros_like(wave)
for sega_tbl in sega_tbls:
    total_time += sega_tbl['DQ_WGT'][0,:]*sega_tbl['EXPTIME']
#xdb.xhist(total_time)

In [42]:
# Find DQmin for all exposures -- Why are we doing this step??
dqmin = np.ones_like(wave).astype(int) * 99999
for sega_tbl in sega_tbls:
    # Reset DQ
    dq = sega_tbl['DQ'][0,:].data
    reset_1024 = dq == 1024
    dq[reset_1024] = 2
    dqmin = np.minimum(dq, dqmin)

In [52]:
# Find DQ_WGT max for all exposures
DQWmax = np.zeros_like(wave)
for sega_tbl in sega_tbls:
    # Reset DQ
    dqw = sega_tbl['DQ_WGT'][0,:].data
    DQWmax = np.maximum(dqw, DQWmax)
#xdb.xhist(dqwmax)

In [55]:
# Total Counts
total_counts = np.zeros_like(wave)
for sega_tbl in sega_tbls:
    # 
    total_counts += DQWmax * sega_tbl['GCOUNTS'][0,:]
xdb.xhist(total_counts)

In [54]:
# Calib

In [56]:
xdb.xplot(wave, total_counts)